In [1]:
# Used Etherscan.io APIs"

In [31]:
import requests
import pandas as pd
import numpy as np
import sys
import time
import os
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm

In [32]:
### Load List of Contracts

In [33]:
contract_old = pd.read_csv("./export-verified-contractaddress-opensource-license_20200207.csv")
contract_new = pd.read_csv("./export-verified-contractaddress-opensource-license_20201002.csv", 
                           skiprows=1)
contracts = pd.concat([contract_new, contract_old], axis=0)
contracts = contracts.drop_duplicates()

In [34]:
# contants
with open("token.txt") as file:  
    API_Token = file.read() 

In [35]:
ADDRESS_ARRAY = contracts.ContractAddress.values

### Create Folders

In [36]:
sol_source = './sol_source'
not_verified = './sol_source_not_verified/'

In [37]:
if not os.path.exists(sol_source):
    os.mkdir(sol_source)
    
if not os.path.exists(not_verified):
    os.mkdir(not_verified)

In [39]:
files_in_sol = os.listdir(sol_source)
files_in_sol = [i.split('_')[0] for i  in files_in_sol]
files_in_not_ver = os.listdir(not_verified)
files_in_not_ver = [i.split('_')[0] for i  in files_in_not_ver]
files_in_sol.extend(files_in_not_ver)
files_in_sol = set(files_in_sol)
not_extracted = list(set(ADDRESS_ARRAY) - files_in_sol)

In [42]:
def scrape_ether_contract_and_write(address_array, API_Token):
    c = 0
    for i, address in tqdm(enumerate(address_array)): 
        # time.sleep(0.01) # we can do 5 GET/POST requests per sec
        url = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={address}&apikey={API_Token}'
        resp = requests.get(url=url)
        data = resp.json()
        try:
            contract_name = data['result'][0]['ContractName']

            # save solidity source code
            with open(f"./sol_source/{address}_{contract_name}.sol", "w", encoding="utf-8") as f:
                print(data['result'][0]['SourceCode'], file=f)

        except:
            c += 1
            time.sleep(0.25)
                    
    print(f'did not work {c} times')

In [44]:
scrape_ether_contract_and_write(address_array=not_extracted, API_Token=API_Token)

14585it [2:35:51,  1.56it/s]

did not work 0 times
